In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/tcc_ceds_music.csv', index_col=0, encoding = 'unicode_escape')

cols = [3, 4, 28]
df = df[df.columns[cols]]
df

,genre,lyrics,topic
0,pop,hold time feel break feel untrue convince spea...,sadness
4,pop,believe drop rain fall grow believe darkest ni...,world/life
6,pop,sweetheart send letter goodbye secret feel bet...,music
10,pop,kiss lips want stroll charm mambo chacha merin...,romantic
12,pop,till darling till matter know till dream live ...,romantic
...,...,...,...
82447,hip hop,cause fuck leave scar tick tock clock come kno...,obscene
82448,hip hop,minks things chain ring braclets yap fame come...,obscene
82449,hip hop,get ban get ban stick crack relax plan attack ...,obscene
82450,hip hop,check check yeah yeah hear thing call switch g...,obscene


In [3]:
len(df)

28372

In [4]:
df.genre.unique()

array(['pop', 'country', 'blues', 'jazz', 'reggae', 'rock', 'hip hop'],
      dtype=object)

In [5]:
import math
df.genre.unique()
train_frames = []
eval_frames = []
for genre in df.genre.unique():
    df_genre = df.loc[df['genre'] == genre]
    train_len = math.floor(len(df_genre) * 9 / 10)
    eval_len = len(df_genre) - train_len
    df_genre_train = df_genre.head(train_len)
    df_genre_eval = df_genre.tail(eval_len)
    train_frames.append(df_genre_train)
    eval_frames.append(df_genre_eval)

train_data = pd.concat(train_frames)
eval_data = pd.concat(eval_frames)

In [7]:
prompt_patterns = ["Generate music lyrics in English for genre {genre}, topic {topic}",
                  "Generate {genre} lyrics about {topic}",
                   "Generate lyrics in genre {genre} about {topic}",
                  "For genre {genre} generate lyrics about {topic}"]

In [8]:
eval_prompts_path = "eval_prompts.txt"
sample_list = []
with open(eval_prompts_path, 'w', encoding="utf-8") as f:
    for index in range(0, eval_data.shape[0]):
        for prompt_pattern in prompt_patterns:
            genre = eval_data.iloc[index]["genre"]
            topic = eval_data.iloc[index]["topic"]
            prompt = prompt_pattern.format(genre = genre, topic = topic)
            f.write(prompt)
            f.write('\n')

In [9]:
input_file_path = "data/input-lyrics.txt"

In [10]:
sample_list = []
with open(input_file_path, 'w', encoding="utf-8") as f:
    for index in range(0, train_data.shape[0]):
        genre = train_data.iloc[index]["genre"]
        lyrics = train_data.iloc[index]["lyrics"]
        topic = train_data.iloc[index]["topic"]
        for prompt_pattern in prompt_patterns:
            prompt = prompt_pattern.format(genre = genre, topic = topic)
            f.write(prompt)
            f.write('\n')
            f.write(lyrics)
            f.write('\n\n')


In [11]:
import os
import requests
import tiktoken
import numpy as np
import pandas as pd

with open(input_file_path, 'r', encoding="utf8") as f:
    data = f.read()
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode with tiktoken gpt2 bpe
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_data)
val_ids = enc.encode_ordinary(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile('data/lyrics/train.bin')
val_ids.tofile('data/lyrics/val.bin')

train has 8,715,109 tokens
val has 971,723 tokens
